<a href="https://colab.research.google.com/github/EllisBuxton/PPIT-Project/blob/main/MusicGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Generator




*   I am going to using a genetic algorithm for the music generation
*   the user will rate each variation of the audio which will alter how much it changes



**The Algorithm:**

In [1]:
from random import choices, randint, randrange, random, sample
from typing import List, Optional, Callable, Tuple

Type Aliases (better maintainability) :

In [2]:
Genome = List[int] # Each genome will represent a specific soloution
Population = List[Genome] # A collection of genome's
PopulateFunc = Callable[[], Population] # Generates initial population of genome's
FitnessFunc = Callable[[Genome], int] # Evaluates how well it solves problem
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]] # Selecting parent genome's based on fitness
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]] #  Takes two genomes as input and returns a tuple of two genomes (used tombine parent genome's)
MutationFunc = Callable[[Genome], Genome] # Takes genome as input and return a modified genome
PrinterFunc = Callable[[Population, int, FitnessFunc], None] # Printing current state of population

Function to generate a radom genome of specified length:

In [3]:
def generate_genome(length: int) -> Genome:
    return choices([0, 1], k=length)

Function to generate a population of genomes :

In [4]:
def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]

Function to perform crossover between two genomes:

In [5]:
def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    # Select a random crossover point
    p = randint(1, length - 1)
    # Perform crossover and return offspring
    return a[0:p] + b[p:], b[0:p] + a[p:]

Function to perform mutation on a genome:

In [6]:
def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        # Flip the bit at the selected index with a certain probability
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome

Function to calculate the total fitness of a population :

In [7]:
def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    return sum([fitness_func(genome) for genome in population])

Function to select a pair of genomes from the population for reproduction:

In [8]:
def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return sample(
        population=generate_weighted_distribution(population, fitness_func),
        k=2
    )

Function to generate a weighted distribution of genomes based on their fitness:

In [9]:
def generate_weighted_distribution(population: Population, fitness_func: FitnessFunc) -> Population:
    result = []

    for gene in population:
        result += [gene] * int(fitness_func(gene)+1)

    return result

Function to sort the population based on fitness:

In [10]:
def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population, key=fitness_func, reverse=True)

Function to convert a genome to a string for printing

In [11]:
def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))

Function to print statistics of the current population

In [12]:
def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]

Main function to run the genetic algorithm:

In [13]:
def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    # Iterate through generations
    for i in range(generation_limit):
        # Sort population based on fitness
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        # Print statistics if printer function is provided
        if printer is not None:
            printer(population, i, fitness_func)

        # Check termination condition
        if fitness_func(population[0]) >= fitness_limit:
            break

        next_generation = population[0:2]

        # Generate next generation
        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i

**Music Generator App code :**

In [ ]:
import click # Used to make a simple interface
from datetime import datetime # Date and time operations
from typing import List, Dict # Type Hinting
from midiutil import MIDIFile # For midi files creation and editing
from pyo import * # Audio synthesis
from algorithms.genetic import generate_genome, Genome, selection_pair, single_point_crossover, mutation # Functions from the algorithm code

Constants for the musical properties:

In [ ]:
BITS_PER_NOTE = 4  # Number of bits representing each note
KEYS = ["C", "C#", "Db", "D", "D#", "Eb", "E", "F", "F#", "Gb", "G", "G#", "Ab", "A", "A#", "Bb", "B"]  # Musical keys
SCALES = ["major", "minorM", "dorian", "phrygian", "lydian", "mixolydian", "majorBlues", "minorBlues"]  # Musical scales

Function converts bits to integers:


In [ ]:
def int_from_bits(bits: List[int]) -> int:
    return int(sum([bit*pow(2, index) for index, bit in enumerate(bits)]))

Function to generate melody from genome:

In [ ]:
def genome_to_melody(genome: Genome, num_bars: int, num_notes: int, num_steps: int,
                     pauses: int, key: str, scale: str, root: int) -> Dict[str, list]:
    # Extracting notes from genome
    notes = [genome[i * BITS_PER_NOTE:i * BITS_PER_NOTE + BITS_PER_NOTE] for i in range(num_bars * num_notes)]

    # Calculating note length
    note_length = 4 / float(num_notes)

    # Creating an event scale
    scl = EventScale(root=key, scale=scale, first=root)

    # Initializing melody dictionary
    melody = {
        "notes": [],
        "velocity": [],
        "beat": []
    }

    # Converting notes to melody events
    for note in notes:
        integer = int_from_bits(note)

        # Handling pauses
        if not pauses:
            integer = int(integer % pow(2, BITS_PER_NOTE - 1))

        # Checking if the note is a rest
        if integer >= pow(2, BITS_PER_NOTE - 1):
            melody["notes"] += [0]
            melody["velocity"] += [0]
            melody["beat"] += [note_length]
        else:
            # Handling sustained notes
            if len(melody["notes"]) > 0 and melody["notes"][-1] == integer:
                melody["beat"][-1] += note_length
            else:
                melody["notes"] += [integer]
                melody["velocity"] += [127]
                melody["beat"] += [note_length]

Function to generate melody from genome:

In [ ]:
def genome_to_melody(genome: Genome, num_bars: int, num_notes: int, num_steps: int,
                     pauses: int, key: str, scale: str, root: int) -> Dict[str, list]:
    # Extracting notes from genome
    notes = [genome[i * BITS_PER_NOTE:i * BITS_PER_NOTE + BITS_PER_NOTE] for i in range(num_bars * num_notes)]

    # Calculating note length
    note_length = 4 / float(num_notes)

    # Creating an event scale
    scl = EventScale(root=key, scale=scale, first=root)

    # Initializing melody dictionary
    melody = {
        "notes": [],
        "velocity": [],
        "beat": []
    }

    # Converting notes to melody events
    for note in notes:
        integer = int_from_bits(note)

        # Handling pauses
        if not pauses:
            integer = int(integer % pow(2, BITS_PER_NOTE - 1))

        # Checking if the note is a rest
        if integer >= pow(2, BITS_PER_NOTE - 1):
            melody["notes"] += [0]
            melody["velocity"] += [0]
            melody["beat"] += [note_length]
        else:
            # Handling sustained notes
            if len(melody["notes"]) > 0 and melody["notes"][-1] == integer:
                melody["beat"][-1] += note_length
            else:
                melody["notes"] += [integer]
                melody["velocity"] += [127]
                melody["beat"] += [note_length]